# 네이버 크롤링으로 알아보는 셀레늄 기초

## 1. 네이버 블로그 공감버튼 클릭하기

In [119]:
# 모듈 임포트(크롬이 설치되어 있어야 합니다.)
#
# %pip install selenium chromedriver_autoinstaller
from selenium import webdriver
import chromedriver_autoinstaller
from selenium.common import NoSuchFrameException, NoSuchElementException
from selenium.webdriver.common.by import By

In [108]:
# 브라우저 열기(셀레늄으로 돌리는 브라우저를 driver라고 부릅니다.)
chromedriver_autoinstaller.install()
driver = webdriver.Chrome()

In [109]:
# 네이버의 차단정책 때문에 로그인은 수동으로 진행해야 합니다.

driver.get("https://naver.com")
###### 수작업으로 네이버 로그인할 것.

내 블로그 아무 포스팅이나 주소를 복사해서 들어가봅시다.

In [75]:
포스팅_주소 = "https://blog.naver.com/pythonrpa/222977573198"
driver.get(포스팅_주소)

In [76]:
# 네이버 블로그는 iframe이라는 태그를 활용합니다.
# iframe 태그 안에는 또 다른 html 파일이 삽입되는데
# iframe 안의 내용은 switch_to.frame 이라는 명령어를 통해
# 내부의 html 안으로 들어간 후에야 접근할 수 있습니다.
#
# 개발자도구를 잘 살펴보면
# 우리가 클릭하고자 하는 공감버튼도 "mainFrame"이라는 프레임 안에 있으므로
# 아래와 같이 mainFrame 안으로 들어갑니다.

driver.switch_to.frame("mainFrame")

In [77]:
# 브라우저에서 해당 공감버튼(하트모양)을 우클릭하고
# "검사"를 선택한 후
# 우측의 개발자도구에서 해당 태그(<a ~ >)에 우클릭하고
# Copy - Copy selector 를 선택하면
# CSS 선택자(CSS Selector)가 클립보드로 복사됩니다.
# 복사된 선택자를 통해 공감버튼 요소를 찾아 클릭합니다.

공감버튼_선택자 = "#area_sympathy222977573198 > div > a"
driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).click()

공감버튼이 잘 클릭되었습니다.

한 가지 아쉬운 점은,

이 공감버튼_선택자 문자열을 다른 포스팅에는 사용할 수 없다는 것입니다.

왜냐면 공감버튼_선택자 중간에 "222977573198"이라는 숫자는

포스팅 고유의 번호이기 때문입니다.

이런 경우를 대비해 CSS선택자는 아래와 같은 문법도 제공하고 있습니다.

"div[id^='area_sympathy'] > div > a"
(id 속성의 값이 "area_sympathy"로 시작하는 div태그 하위의 div태그 하위의 a태그)

이 선택자로 코드를 바꾸면 됩니다.

동작은 동일하지만, 한 가지 바뀐 점은

다른 어느 블로그나 어느 포스팅에 들어가더라도

공감버튼을 찾을 수 있게 되었다는 것입니다.

In [ ]:
공감버튼_선택자 = "div[id^='area_sympathy'] > div > a"
driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).click()

한 가지만 더 보완합시다.

공감버튼이 이미 클릭되어 있던 경우라면?

이 코드를 실행했을 때 공감이 해제됩니다.

그래서 if-else 구문을 넣어

공감이 눌려 있지 않은 상태에서만

공감을 클릭하도록 분기를 만들어야 합니다.

공감이 눌러진 상태와 그렇지 않은 상태의 차이를

개발자도구를 열어 가만히 살펴보면

우리가 클릭하고 있는 a태그의 aria-pressed 값이 "true" / "false"로

토글되는 것을 확인할 수 있습니다.

a태그의 aria-pressed 속성값을 확인하는 문법은 아래와 같습니다.

In [78]:
driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).get_attribute("aria-pressed")

'true'

현재는 공감이 눌러져 있어 "true"를 리턴했습니다.

이런 경우에는 자동클릭을 하지 않아야 하므로 아래와 같이

if문을 넣어 코드를 수정합니다.

In [79]:
if driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).get_attribute("aria-pressed") == "false":
    driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).click()

여러 번 실행해도 공감이 해제되지 않는 것을 확인하였습니다.

이제 이 코드의 편한 재사용을 위해 함수를 만들어 둡시다.

In [83]:
def 공감_클릭():
    """
    블로그 포스팅 내에 들어간 상태로 실행해야 함
    """
    try:
        driver.switch_to.frame("mainFrame")
    except NoSuchFrameException as e:
        pass
    공감버튼_선택자 = "div[id^='area_sympathy'] > div > a"
    if driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).get_attribute("aria-pressed") == "false":
        driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).click()
    driver.switch_to.parent_frame()

In [84]:
# 다른 글로 이동 후 아래 함수 실행
공감_클릭()

다른 포스팅에 들어가서도 공감_클릭 함수가 작동하는지 확인해봅시다.

잘 작동한다면 이제 다음 챕터로 넘어가겠습니다.

# 2. 블로그 메인화면에서 이웃새글 10개 공감 클릭하는 코드

![](https://i.ibb.co/bzTSJJJ/2023-06-28-141211.png)

네이버블로그 홈화면인 blog.naver.com에 접속하면 하단에 위 이미지와 같이

이웃들의 새 글이 추천됩니다.

인플루언서 또는 파워블로거들과 이웃이 되거나

공감, 댓글을 남기는 행동을 통해 상대방이 내 블로그에 방문할 수 있고,

결과적으로 내 블로그의 지수를 끌어올릴 수 있게 됩니다.

이번 챕터에서는 간단히

이웃새글 섹션의 10개 추천글에 공감을 누르는 코드를 작성해보겠습니다.

In [97]:
# 블로그 홈으로 이동
driver.get("https://blog.naver.com")

# 이웃새글 섹션의 공감버튼 10개 탐색하기
블로그메인_공감버튼_선택자 = "#content > section > div.list_post_article.list_post_article_comments > div > div > div.info_post > div.comments > div > div > a"

공감리스트 = driver.find_elements(By.CSS_SELECTOR, 블로그메인_공감버튼_선택자)

In [98]:
# for문으로 공감버튼 10개 전부 클릭하기
for 하트 in 공감리스트:
    if 하트.get_attribute("aria-pressed") == "false":
        하트.click()

![](https://i.ibb.co/B6Mf8KZ/2023-06-28-142729.png)

메인화면의 10개 포스팅 중 공감버튼이 있는 9개의 공감버튼을

순식간에 클릭하였습니다.

# 3. 추천포스팅 하나하나 직접 들어가서 공감 클릭하고 나오기

이번에는 (약간 다른 방식으로) 추천포스팅에 하나씩 직접 들어가서 공감을 클릭해보겠습니다. 공감을 누르고 "좋은 글 남겨주셔서 감사합니다. 공감 꾹 누르고 갑니다^^" 라는 훈훈한 댓글을 남겨놓으면 우리 블로그로 유입되는 트래픽도 유의미하게 증가할 것입니다.

이번에는 특정 태그를 클릭하는 방식이 아니라, 해당 포스팅의 주소를 추출해서 직접 driver.get()을 통해 접근하는 방식으로 진행하겠습니다. 우선 첫 번째 글의 제목을 우클릭하고 "검사"를 선택해서 a태그의 CSS선택자를 복사합니다.

![](https://i.ibb.co/GdgTjyz/2023-06-28-144533.png)

복사한 선택자는 아래와 같습니다.

In [41]:
첫번째글_선택자 = "#content > section > div.list_post_article.list_post_article_comments > div:nth-child(1) > div > div.info_post > div.desc > a.desc_inner"

# 두 번째 글의 CSS선택자도 복사해봅니다.
두번째글_선택자 = "#content > section > div.list_post_article.list_post_article_comments > div:nth-child(2) > div > div.info_post > div.desc > a.desc_inner"

위의 첫번째글_선택자와 두번째글_선택자의 차이를 살펴보니까

가운데쯤에 "div:nth-child(#)" 이 부분만 다르고 나머지는 동일합니다.

그래서 ":nth-child(#)" 부분을 지우면 10개 글이 전부 선택되는 선택자가 됩니다.

그래서 아래와 같이 수정 후 진행해보겠습니다.

In [99]:
글제목_선택자 = "#content > section > div.list_post_article.list_post_article_comments > div > div > div.info_post > div.desc > a.desc_inner"

글_리스트 = driver.find_elements(By.CSS_SELECTOR, 글제목_선택자)

In [100]:
len(글_리스트)

10

10개의 태그가 추출된 걸 보니 선택자는 잘 작성된 것 같네요.

그럼 각 요소에서 href 속성값인 "https://blog.naver.com/아이디/글번호" 문자열을

리스트 컴프리헨션을 통해 추출해보겠습니다.

In [101]:
주소_리스트 = [i.get_attribute("href") for i in 글_리스트]
주소_리스트

['https://blog.naver.com/ridesafe/223140298862',
 'https://blog.naver.com/ferieo/223139205539',
 'https://blog.naver.com/annerheekim/223140293036',
 'https://blog.naver.com/gyoil/223140200657',
 'https://blog.naver.com/moeblog/223139943528',
 'https://blog.naver.com/ferieo/223139185729',
 'https://blog.naver.com/moeblog/223140062473',
 'https://blog.naver.com/aiwositai/223140037149',
 'https://blog.naver.com/moeblog/223140028008',
 'https://blog.naver.com/moeblog/223139871222']

10개 포스팅의 주소와 공감_클릭() 함수가 있으니 이제 실행만 해 보면 되겠네요.

잘 되는지 테스트해봅시다.

In [102]:
for i in 주소_리스트:
    driver.get(i)
    공감_클릭()

![](https://i.ibb.co/dKVxK9t/2023-06-28-145917.png)

위와 같이 10개 포스팅의 공감 버튼을 모두 클릭하고 작업이 종료되었습니다.

# 4. 이웃새글 모든 페이지의 포스팅 전부 공감 클릭하기

아직까지 우리는 1페이지의 10개 글에만 공감을 클릭했습니다.

하지만 셀레늄 크롤링의 진정한 가치는

어마어마한 반복작업을 실행할 때 비로소 느낄 수 있습니다.

이번에는 페이지를 전부 탐색하면서 주소_리스트를 추가하고

공감_클릭()을 실행해보겠습니다.

먼저 1페이지에서 주소창을 잘 살펴봅시다.

![](https://i.ibb.co/8xCTLCf/2023-06-28-150522.png)

이번엔 2페이지로 이동한 후 주소창의 URL을 다시 살펴봅시다.

![](https://i.ibb.co/JqLbPhz/2023-06-28-150624.png)

차이가 보이시죠? 주소창의 "currentPage={}" 이 부분만 바꿔 입력하면

이웃새글 모든 페이지를 탐색할 수 있을 것 같습니다.

제 경우에는 마지막 페이지가 18페이지입니다.

여러분도 각각 마지막 페이지가 다를 것입니다.

그래서 마지막 페이지를 확인해보려면

직접 currentPage={} 부분에 1부터 넣어보면서

접속이 되는지 여부만 확인하면 될 것 같습니다.

19페이지로 접속해보니 아래와 같은 메시지만 뜹니다.

![](https://i.ibb.co/PCcKDYB/2023-06-28-150918.png)

## 아하!

while문으로 페이지번호를 1씩 올리면서 접속해보다가

html소스에 "업데이트된 이웃의 새 글이 없습니다" 라는 문자열이 있으면

그 때 while문을 종료하면 되겠군요.

또 다른 while문 종료방법은,

페이지의 글_리스트의 길이가 0일 때 종료하는 방법입니다.

(이 방법을 사용하겠습니다.)

그럼 이 로직을 코드로 옮겨봅시다.

In [114]:
from time import sleep

글제목_선택자 = "#content > section > div.list_post_article.list_post_article_comments > div > div > div.info_post > div.desc > a.desc_inner"

페이지번호 = 1
주소리스트 = []

while True:
    driver.get(f"https://section.blog.naver.com/BlogHome.naver?directoryNo=0&currentPage={페이지번호}&groupId=0")
    sleep(1)  # 페이지 로딩이 완료되기 전에 넘어가는 경향이 있어 1초 쉬어줌
    글_리스트 = driver.find_elements(By.CSS_SELECTOR, 글제목_선택자)
    if not 글_리스트:
        break
    주소리스트 += [i.get_attribute("href") for i in 글_리스트]
    페이지번호 += 1

In [117]:
len(주소리스트)

171

171개의 포스팅 주소가 모였습니다.

그럼 챕터3에서 작성했던 코드를(사실 세 줄 밖에 안 됩니다.)

그대로 실행하여 171개의 포스팅에 공감을 눌러봅시다.

In [118]:
for i in 주소리스트:
    driver.get(i)
    공감_클릭()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[id^='area_sympathy'] > div > a"}
  (Session info: chrome=114.0.5735.198); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DEA813+48355]
	(No symbol) [0x00D7C4B1]
	(No symbol) [0x00C85358]
	(No symbol) [0x00CB09A5]
	(No symbol) [0x00CB0B3B]
	(No symbol) [0x00CDE232]
	(No symbol) [0x00CCA784]
	(No symbol) [0x00CDC922]
	(No symbol) [0x00CCA536]
	(No symbol) [0x00CA82DC]
	(No symbol) [0x00CA93DD]
	GetHandleVerifier [0x0104AABD+2539405]
	GetHandleVerifier [0x0108A78F+2800735]
	GetHandleVerifier [0x0108456C+2775612]
	GetHandleVerifier [0x00E751E0+616112]
	(No symbol) [0x00D85F8C]
	(No symbol) [0x00D82328]
	(No symbol) [0x00D8240B]
	(No symbol) [0x00D74FF7]
	BaseThreadInitThunk [0x760C00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77A87

아차! 오류가 발생하네요?

공감버튼이 없는 페이지가 있기 때문입니다.

공감버튼이 없을 때 오류를 내지 말고 패스할 수 있게

공감_클릭() 함수를 조금만 수정해줍시다. (try-except문을 사용합니다.)

In [121]:
def 공감_클릭2():
    """
    블로그 포스팅 내에 들어간 상태로 실행해야 함
    """
    try:
        driver.switch_to.frame("mainFrame")
    except NoSuchFrameException as e:
        pass
    공감버튼_선택자 = "div[id^='area_sympathy'] > div > a"
    try:
        if driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).get_attribute("aria-pressed") == "false":
            driver.find_element(By.CSS_SELECTOR, 공감버튼_선택자).click()
    except NoSuchElementException as e:
        pass
    driver.switch_to.parent_frame()

In [ ]:
%pip install tqdm
from tqdm import tqdm  # 반복문 실행시 진행상태를 표시해주는 모듈

In [123]:
# 수정된 공감_클릭2 함수로 재실행
for i in tqdm(주소리스트):
    driver.get(i)
    공감_클릭2()

                                              0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


100%|██████████| 171/171 [04:22<00:00,  1.54s/it]


171개의 이웃새글 포스팅을 돌면서 공감을 다 눌렀습니다.

약 4분 정도 걸렸네요.

이렇게 오래 걸리는 작업은 무작정 넋놓고 기다리기보다는

(enumerate를 써서 for문 안에서 인덱스를 출력하는 방법도 있지만)

위처럼 tqdm이라는 모듈을 사용해서

현재 진행정도와 예상시간을 출력할 수도 있습니다.

셀레늄은 제 과정 말미에 한 번 더 (며칠에 걸쳐)

다양한 사이트의 데이터나 첨부파일을 다운받아보는 연습을 해볼 예정입니다.

파이썬-셀레늄 관련한 수많은 포스팅이 있으니

참고하셔서 여러분에게 필요한 멋진 크롤링 프로그램을 완성해 보시기 바랍니다.

# 끝.